
An attempt to measure suprise in text using adapters

https://github.com/huggingface/peft/blob/main/examples/fp4_finetuning/finetune_fp4_opt_bnb_peft.py

In [1]:
import os
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import numpy as np
import pandas as pd

/media/wassname/SGIronWolf/projects5/bs_writing_detector/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import os

import torch
import torch.nn as nn
import transformers
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

# from peft import LoraConfig, get_peft_model


# os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [3]:
MAX_LEN = 2000
import json
samples = json.load(open("../samples.json"))
df_samples = pd.DataFrame(samples)
df_samples['len'] = df_samples['text'].str.len()
df_samples


sample = samples[0]
sample

{'name': 'bad_ml',
 'url': 'https://arxiv.org/abs/2312.10868',
 'text': 'This roadmap survey has embarked on an exploration of the\ntransformative trends in generative AI research, particularly focusing on speculated advancements like Q* and the progressive strides towards AGI. Our analysis highlights a crucial paradigm shift, driven by innovations such as MoE, multi-modal learning, and the pursuit of AGI. These advancements signal a future where AI systems could significantly extend their capabilities in reasoning, contextual understanding, and creative problem-solving. This study reflects on AI’s dual potential to either contribute to or impede global equity and justice. The equitable distribution of AI benefits and its role in decision-making processes raise crucial questions about fairness and inclusivity. It is imperative to thoughtfully integrate AI into societal structures to enhance justice and reduce disparities. Despite these advancements, several open questions and research 

## Helpers

In [4]:
from dotenv import load_dotenv

load_dotenv() 

True

In [5]:
import os
from openai import OpenAI

def summize(text):
    client = OpenAI()
    content = f"Make a tl;dr of this text in <280 chars.\n\n## Text\n\n{text}\n\n## Instruction\n\nMake a tl;dr of this text in <280 chars. Start with the most important, as extra text will be discarded :\n\ntl;dr:"
    chat_completion = client.chat.completions.create(
            messages=[
                {
                    "role": "user",
                    "content": content,
                }
            ],
            model="gpt-4",
        )
    # print(content)
    r = chat_completion.choices[0].message.content
    return r

r = summize(samples[1]["text"])
r

'Exploring representation engineering (RepE) for AI transparency, we proposed new methods that improved results on TruthfulQA and presented a tool for AI safety issues. Future RepE research could further boost our control over AI systems.'

In [6]:
# modified from https://github.dev/huggingface/evaluate/blob/8dfe05784099fb9af55b8e77793205a3b7c86465/measurements/perplexity/perplexity.py#L154

# from evaluate.measurements.perplexity import Perplexity
import evaluate
from evaluate import logging
from torch.nn import CrossEntropyLoss

# @evaluate.utils.file_utils.add_start_docstrings(_DESCRIPTION, _KWARGS_DESCRIPTION)
def perplexity_compute(
    data, model, tokenizer, batch_size: int = 16, add_start_token: bool = True, device=None, max_length=None
):

    if device is not None:
        assert device in ["gpu", "cpu", "cuda"], "device should be either gpu or cpu."
        if device == "gpu":
            device = "cuda"
    else:
        device = "cuda" if torch.cuda.is_available() else "cpu"

    # model = AutoModelForCausalLM.from_pretrained(model_id)
    model = model.to(device)

    # tokenizer = AutoTokenizer.from_pretrained(model_id)

    # if batch_size > 1 (which generally leads to padding being required), and
    # if there is not an already assigned pad_token, assign an existing
    # special token to also be the padding token
    if tokenizer.pad_token is None and batch_size > 1:
        existing_special_tokens = list(tokenizer.special_tokens_map_extended.values())
        # check that the model already has at least one special token defined
        assert (
            len(existing_special_tokens) > 0
        ), "If batch_size > 1, model must have at least one special token to use for padding. Please use a different model or set batch_size=1."
        # assign one of the special tokens to also be the pad token
        tokenizer.add_special_tokens({"pad_token": existing_special_tokens[0]})

    if add_start_token and max_length:
        # leave room for <BOS> token to be added:
        assert (
            tokenizer.bos_token is not None
        ), "Input model must already have a BOS token if using add_start_token=True. Please use a different model, or set add_start_token=False"
        max_tokenized_len = max_length - 1
    else:
        max_tokenized_len = max_length

    encodings = tokenizer(
        data,
        add_special_tokens=False,
        padding=True,
        truncation=True if max_tokenized_len else False,
        max_length=max_tokenized_len,
        return_tensors="pt",
        return_attention_mask=True,
    ).to(device)

    encoded_texts = encodings["input_ids"]
    attn_masks = encodings["attention_mask"]

    # check that each input is long enough:
    if add_start_token:
        assert torch.all(torch.ge(attn_masks.sum(1), 1)), "Each input text must be at least one token long."
    else:
        assert torch.all(
            torch.ge(attn_masks.sum(1), 2)
        ), "When add_start_token=False, each input text must be at least two tokens long. Run with add_start_token=True if inputting strings of only one token, and remove all empty input strings."

    ppls = []
    loss_fct = CrossEntropyLoss(reduction="none")

    for start_index in logging.tqdm(range(0, len(encoded_texts), batch_size)):
        end_index = min(start_index + batch_size, len(encoded_texts))
        encoded_batch = encoded_texts[start_index:end_index]
        attn_mask = attn_masks[start_index:end_index]

        if add_start_token:
            bos_tokens_tensor = torch.tensor([[tokenizer.bos_token_id]] * encoded_batch.size(dim=0)).to(device)
            encoded_batch = torch.cat([bos_tokens_tensor, encoded_batch], dim=1)
            attn_mask = torch.cat(
                [torch.ones(bos_tokens_tensor.size(), dtype=torch.int64).to(device), attn_mask], dim=1
            )

        labels = encoded_batch

        with torch.no_grad():
            out_logits = model(encoded_batch, attention_mask=attn_mask).logits

        shift_logits = out_logits[..., :-1, :].contiguous()
        shift_labels = labels[..., 1:].contiguous()
        shift_attention_mask_batch = attn_mask[..., 1:].contiguous()

        perplexity_batch = torch.exp(
            (loss_fct(shift_logits.transpose(1, 2), shift_labels) * shift_attention_mask_batch).sum(1)
            / shift_attention_mask_batch.sum(1)
        )

        ppls += perplexity_batch.tolist()

    return {"perplexities": ppls, "mean_perplexity": np.mean(ppls)}

In [7]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoModelForMaskedLM, AutoModelForCausalLM, AutoConfig, PreTrainedTokenizerBase, PreTrainedTokenizer, GPTQConfig, BitsAndBytesConfig

def load_model(model_name):
    trust_remote_code = True
    tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=trust_remote_code)
    config = AutoConfig.from_pretrained(model_name, trust_remote_code=trust_remote_code)
    # print(config)
    if config.quantization_config is not None:
        config.quantization_config['disable_exllama'] = True
        if 'use_exllama' in config.quantization_config:
            del config.quantization_config['use_exllama']
    model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=trust_remote_code, 
                                                 config=config,
                                                 )
    return model, tokenizer


In [8]:
import gc

def clear_mem():
    torch.cuda.empty_cache()
    gc.collect()

## Results

In [9]:
models = [
    "TheBloke/phi-2-GPTQ",
    "TheBloke/Llama-2-7B-GPTQ",
    "TheBloke/Llama-2-13B-GPTQ",
    "TheBloke/Mistral-7B-v0.1-GPTQ",
]


In [13]:
summaries = {}
data = []
for model_name in models:
    model, tokenizer = load_model(model_name)
    for sample in samples:
        if sample['name'] not in summaries:
            summaries[sample['name']] = summize(sample['text'])[:600]
        summary = summaries[sample['name']]

        # before 
        s1 = sample['text']
        results = perplexity_compute(data=s1, model=model, tokenizer=tokenizer, device='cuda')
        before = results['mean_perplexity']

        # after 
        s2 = f"""
        High level summary: {summary}

Text:
{sample['text']}
        """
        results = perplexity_compute(data=s2, model=model, tokenizer=tokenizer, device='cuda')
        after = np.array(results['perplexities'])[-len(s1):].mean()

        print(model_name, sample['name'], before, after)
        data.append(dict(before=before, after=after, model=model_name, sample=sample['name'],
                         in_training=sample['in_training'], len=sample['len']))


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Using pad_token, but it is not set yet.
100%|██████████| 1/1 [00:00<00:00,  4.14it/s]


TheBloke/phi-2-GPTQ bad_ml 12.456705093383789 11.44649887084961


100%|██████████| 1/1 [00:00<00:00,  4.92it/s]


TheBloke/phi-2-GPTQ good_ml 22.6639461517334 20.115413665771484


100%|██████████| 1/1 [00:00<00:00,  3.81it/s]


...

TheBloke/Mistral-7B-v0.1-GPTQ email_to_fauci 10.894938468933105 10.550251960754395


100%|██████████| 1/1 [00:00<00:00,  2.51it/s]


TheBloke/Mistral-7B-v0.1-GPTQ enron_email1 12.993982315063477 11.092223167419434


100%|██████████| 1/1 [00:00<00:00,  2.16it/s]

TheBloke/Mistral-7B-v0.1-GPTQ openai_board_ann 5.521510124206543 6.3877854347229


In [33]:
# results
df = pd.DataFrame(data)
df["learning%"] = (df["before"] - df["after"])/df["before"]
df['in_training'] = None
# df

In [34]:
df

,before,after,model,sample,learning%,in_training
0,12.456705,11.446499,TheBloke/phi-2-GPTQ,bad_ml,0.081097,None
1,22.663946,20.115414,TheBloke/phi-2-GPTQ,good_ml,0.112449,None
2,14.285429,14.216052,TheBloke/phi-2-GPTQ,sokal hoax,0.004856,None
3,20.507641,19.644333,TheBloke/phi-2-GPTQ,Theory o. general relativity,0.042097,None
4,1.164225,2.354216,TheBloke/phi-2-GPTQ,lorem ipsum,-1.022131,None
5,18.052301,14.710425,TheBloke/phi-2-GPTQ,wikipedia on LK-99,0.185122,None
6,2.836233,4.256137,TheBloke/phi-2-GPTQ,I have a dream,-0.500631,None
7,7.091303,7.495458,TheBloke/phi-2-GPTQ,AI gen fake paper,-0.056993,None
8,28.309990,27.353872,TheBloke/phi-2-GPTQ,"Schmidhuber 2023 Subjective Novelty, Surprise",0.033773,None
9,21.060320,18.637592,TheBloke/phi-2-GPTQ,email_to_fauci,0.115038,None


In [35]:
for n,d in df.groupby("model"):
    print(n)
    d = d[['sample', 'learning%', 'in_training']].sort_values("learning%", ascending=False)
    print(d.to_markdown())
    display(d)

TheBloke/Llama-2-13B-GPTQ
|    | sample                                        |    learning% | in_training   |
|---:|:----------------------------------------------|-------------:|:--------------|
| 29 | wikipedia on LK-99                            |  0.167259    |               |
| 34 | enron_email1                                  |  0.11515     |               |
| 33 | email_to_fauci                                |  0.0738711   |               |
| 25 | good_ml                                       |  0.0602946   |               |
| 31 | AI gen fake paper                             |  0.0191481   |               |
| 24 | bad_ml                                        |  0.00543562  |               |
| 32 | Schmidhuber 2023 Subjective Novelty, Surprise | -0.000410921 |               |
| 27 | Theory o. general relativity                  | -0.0443644   |               |
| 35 | openai_board_ann                              | -0.0662883   |               |
| 26 | sokal hoax           

,sample,learning%,in_training
29,wikipedia on LK-99,0.167259,None
34,enron_email1,0.115150,None
33,email_to_fauci,0.073871,None
25,good_ml,0.060295,None
31,AI gen fake paper,0.019148,None
24,bad_ml,0.005436,None
32,"Schmidhuber 2023 Subjective Novelty, Surprise",-0.000411,None
27,Theory o. general relativity,-0.044364,None
35,openai_board_ann,-0.066288,None
26,sokal hoax,-0.162956,None


TheBloke/Llama-2-7B-GPTQ
|    | sample                                        |   learning% | in_training   |
|---:|:----------------------------------------------|------------:|:--------------|
| 17 | wikipedia on LK-99                            |  0.202426   |               |
| 22 | enron_email1                                  |  0.190753   |               |
| 21 | email_to_fauci                                |  0.0967749  |               |
| 13 | good_ml                                       |  0.086104   |               |
| 12 | bad_ml                                        |  0.0446809  |               |
| 19 | AI gen fake paper                             |  0.0185693  |               |
| 15 | Theory o. general relativity                  |  0.0132304  |               |
| 20 | Schmidhuber 2023 Subjective Novelty, Surprise |  0.0116207  |               |
| 23 | openai_board_ann                              |  0.00617345 |               |
| 14 | sokal hoax                       

,sample,learning%,in_training
17,wikipedia on LK-99,0.202426,None
22,enron_email1,0.190753,None
21,email_to_fauci,0.096775,None
13,good_ml,0.086104,None
12,bad_ml,0.044681,None
19,AI gen fake paper,0.018569,None
15,Theory o. general relativity,0.013230,None
20,"Schmidhuber 2023 Subjective Novelty, Surprise",0.011621,None
23,openai_board_ann,0.006173,None
14,sokal hoax,-0.061205,None


TheBloke/Mistral-7B-v0.1-GPTQ
|    | sample                                        |   learning% | in_training   |
|---:|:----------------------------------------------|------------:|:--------------|
| 46 | enron_email1                                  |  0.146357   |               |
| 41 | wikipedia on LK-99                            |  0.11695    |               |
| 37 | good_ml                                       |  0.0815578  |               |
| 45 | email_to_fauci                                |  0.0316373  |               |
| 43 | AI gen fake paper                             |  0.00915127 |               |
| 44 | Schmidhuber 2023 Subjective Novelty, Surprise | -0.00355543 |               |
| 36 | bad_ml                                        | -0.00760339 |               |
| 38 | sokal hoax                                    | -0.0617226  |               |
| 39 | Theory o. general relativity                  | -0.0846206  |               |
| 47 | openai_board_ann            

,sample,learning%,in_training
46,enron_email1,0.146357,None
41,wikipedia on LK-99,0.116950,None
37,good_ml,0.081558,None
45,email_to_fauci,0.031637,None
43,AI gen fake paper,0.009151,None
44,"Schmidhuber 2023 Subjective Novelty, Surprise",-0.003555,None
36,bad_ml,-0.007603,None
38,sokal hoax,-0.061723,None
39,Theory o. general relativity,-0.084621,None
47,openai_board_ann,-0.156891,None


TheBloke/phi-2-GPTQ
|    | sample                                        |   learning% | in_training   |
|---:|:----------------------------------------------|------------:|:--------------|
|  5 | wikipedia on LK-99                            |  0.185122   |               |
| 10 | enron_email1                                  |  0.138411   |               |
|  9 | email_to_fauci                                |  0.115038   |               |
|  1 | good_ml                                       |  0.112449   |               |
|  0 | bad_ml                                        |  0.0810974  |               |
|  3 | Theory o. general relativity                  |  0.0420969  |               |
|  8 | Schmidhuber 2023 Subjective Novelty, Surprise |  0.0337732  |               |
|  2 | sokal hoax                                    |  0.00485648 |               |
|  7 | AI gen fake paper                             | -0.056993   |               |
| 11 | openai_board_ann                      

,sample,learning%,in_training
5,wikipedia on LK-99,0.185122,None
10,enron_email1,0.138411,None
9,email_to_fauci,0.115038,None
1,good_ml,0.112449,None
0,bad_ml,0.081097,None
3,Theory o. general relativity,0.042097,None
8,"Schmidhuber 2023 Subjective Novelty, Surprise",0.033773,None
2,sokal hoax,0.004856,None
7,AI gen fake paper,-0.056993,None
11,openai_board_ann,-0.059498,None


In [36]:
df["learning%"] = (df["before"] - df["after"])
for n,d in df.groupby("model"):
    print(n)
    d = d[['sample', 'learning%', 'in_training']].sort_values("learning%", ascending=False)
    print(d.to_markdown())
    display(d)

TheBloke/Llama-2-13B-GPTQ
|    | sample                                        |   learning% | in_training   |
|---:|:----------------------------------------------|------------:|:--------------|
| 29 | wikipedia on LK-99                            |  1.94887    |               |
| 34 | enron_email1                                  |  1.43055    |               |
| 25 | good_ml                                       |  0.753304   |               |
| 33 | email_to_fauci                                |  0.726738   |               |
| 31 | AI gen fake paper                             |  0.106177   |               |
| 24 | bad_ml                                        |  0.041275   |               |
| 32 | Schmidhuber 2023 Subjective Novelty, Surprise | -0.00523758 |               |
| 35 | openai_board_ann                              | -0.355879   |               |
| 27 | Theory o. general relativity                  | -0.526403   |               |
| 26 | sokal hoax                      

,sample,learning%,in_training
29,wikipedia on LK-99,1.948873,None
34,enron_email1,1.430546,None
25,good_ml,0.753304,None
33,email_to_fauci,0.726738,None
31,AI gen fake paper,0.106177,None
24,bad_ml,0.041275,None
32,"Schmidhuber 2023 Subjective Novelty, Surprise",-0.005238,None
35,openai_board_ann,-0.355879,None
27,Theory o. general relativity,-0.526403,None
26,sokal hoax,-0.593387,None


TheBloke/Llama-2-7B-GPTQ
|    | sample                                        |   learning% | in_training   |
|---:|:----------------------------------------------|------------:|:--------------|
| 22 | enron_email1                                  |   2.6292    |               |
| 17 | wikipedia on LK-99                            |   2.49834   |               |
| 13 | good_ml                                       |   1.24296   |               |
| 21 | email_to_fauci                                |   1.09521   |               |
| 12 | bad_ml                                        |   0.357408  |               |
| 15 | Theory o. general relativity                  |   0.182519  |               |
| 20 | Schmidhuber 2023 Subjective Novelty, Surprise |   0.158172  |               |
| 19 | AI gen fake paper                             |   0.114477  |               |
| 23 | openai_board_ann                              |   0.0365281 |               |
| 14 | sokal hoax                       

,sample,learning%,in_training
22,enron_email1,2.629203,None
17,wikipedia on LK-99,2.498336,None
13,good_ml,1.242956,None
21,email_to_fauci,1.095209,None
12,bad_ml,0.357408,None
15,Theory o. general relativity,0.182519,None
20,"Schmidhuber 2023 Subjective Novelty, Surprise",0.158172,None
19,AI gen fake paper,0.114477,None
23,openai_board_ann,0.036528,None
14,sokal hoax,-0.454623,None


TheBloke/Mistral-7B-v0.1-GPTQ
|    | sample                                        |   learning% | in_training   |
|---:|:----------------------------------------------|------------:|:--------------|
| 46 | enron_email1                                  |   1.90176   |               |
| 41 | wikipedia on LK-99                            |   1.34696   |               |
| 37 | good_ml                                       |   1.24542   |               |
| 45 | email_to_fauci                                |   0.344687  |               |
| 43 | AI gen fake paper                             |   0.0518088 |               |
| 44 | Schmidhuber 2023 Subjective Novelty, Surprise |  -0.0493546 |               |
| 36 | bad_ml                                        |  -0.0642319 |               |
| 38 | sokal hoax                                    |  -0.346581  |               |
| 47 | openai_board_ann                              |  -0.866275  |               |
| 39 | Theory o. general relativity

,sample,learning%,in_training
46,enron_email1,1.901759,None
41,wikipedia on LK-99,1.346963,None
37,good_ml,1.245416,None
45,email_to_fauci,0.344687,None
43,AI gen fake paper,0.051809,None
44,"Schmidhuber 2023 Subjective Novelty, Surprise",-0.049355,None
36,bad_ml,-0.064232,None
38,sokal hoax,-0.346581,None
47,openai_board_ann,-0.866275,None
39,Theory o. general relativity,-1.019134,None


TheBloke/phi-2-GPTQ
|    | sample                                        |   learning% | in_training   |
|---:|:----------------------------------------------|------------:|:--------------|
|  5 | wikipedia on LK-99                            |   3.34188   |               |
| 10 | enron_email1                                  |   3.16623   |               |
|  1 | good_ml                                       |   2.54853   |               |
|  9 | email_to_fauci                                |   2.42273   |               |
|  0 | bad_ml                                        |   1.01021   |               |
|  8 | Schmidhuber 2023 Subjective Novelty, Surprise |   0.956118  |               |
|  3 | Theory o. general relativity                  |   0.863308  |               |
|  2 | sokal hoax                                    |   0.0693769 |               |
|  7 | AI gen fake paper                             |  -0.404154  |               |
| 11 | openai_board_ann                      

,sample,learning%,in_training
5,wikipedia on LK-99,3.341876,None
10,enron_email1,3.166235,None
1,good_ml,2.548532,None
9,email_to_fauci,2.422728,None
0,bad_ml,1.010206,None
8,"Schmidhuber 2023 Subjective Novelty, Surprise",0.956118,None
3,Theory o. general relativity,0.863308,None
2,sokal hoax,0.069377,None
7,AI gen fake paper,-0.404154,None
11,openai_board_ann,-0.508879,None


# TODO: compare big and small

In [42]:
a = df[df.model=="TheBloke/Llama-2-7B-GPTQ"].set_index('sample').drop(columns=['model', 'in_training'])
b = df[df.model=="TheBloke/Llama-2-13B-GPTQ"].set_index('sample').drop(columns=['model', 'in_training'])
d = (a-b).sort_values("after", ascending=False)
print('big numbers (for after and learning) mean the smaller model was more confused')
print(d.to_markdown())
d


big numbers mean a difference between big and small models
| sample                                        |   before |      after |   learning% |
|:----------------------------------------------|---------:|-----------:|------------:|
| sokal hoax                                    | 3.7865   |  3.64774   |   0.138764  |
| good_ml                                       | 1.94178  |  1.45213   |   0.489652  |
| Theory o. general relativity                  | 1.92992  |  1.221     |   0.708922  |
| email_to_fauci                                | 1.47915  |  1.11068   |   0.368471  |
| Schmidhuber 2023 Subjective Novelty, Surprise | 0.865227 |  0.701818  |   0.163409  |
| AI gen fake paper                             | 0.619803 |  0.611503  |   0.0083003 |
| I have a dream                                | 0.317971 |  0.212919  |   0.105052  |
| enron_email1                                  | 1.35998  |  0.161322  |   1.19866   |
| openai_board_ann                              | 0.548308 | 

,before,after,learning%
sample,,,
sokal hoax,3.786502,3.647738,0.138764
good_ml,1.941784,1.452132,0.489652
Theory o. general relativity,1.929925,1.221003,0.708922
email_to_fauci,1.479151,1.110680,0.368471
"Schmidhuber 2023 Subjective Novelty, Surprise",0.865227,0.701818,0.163409
AI gen fake paper,0.619803,0.611503,0.008300
I have a dream,0.317971,0.212919,0.105052
enron_email1,1.359979,0.161322,1.198657
openai_board_ann,0.548308,0.155900,0.392407
